<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City - Part 2</font></h1>

## 1. Download required libraries

In [76]:
!conda install -c conda-forge beautifulsoup4 lxml html5lib requests geocoder geopy folium --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4
    - geocoder
    - html5lib
    - lxml
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW 

## 2. Download html page and put in a soup object

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

## 3. Scrape html page and save data into an array

In [ ]:
# Using google location
import geocoder # import geocoder

table = soup.find('table',{'class':'wikitable sortable'})
listdict = []
for row in table.find_all('tr'):
    elements = row.find_all('td')
    if not elements:
        continue
    postalCode, borough, neighborhood = [element.text.strip() for element in elements]
    if borough != 'Not assigned':
        # initialize your variable to None
        lat_lng_coords = None
        
        # loop until you get the coordinates
        while(lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(postalCode))
            lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        if neighborhood != 'Not assigned':
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':neighborhood, 'Latitude': latitude, 'Longitude': longitude})
        else:
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':borough, 'Latitude': latitude, 'Longitude': longitude})

In [2]:
# Using Geospatial_Coordinates.csv
table = soup.find('table',{'class':'wikitable sortable'})
listdict = []
coordinates = pd.read_csv('https://cocl.us/Geospatial_data', delimiter = ',')
coordinates.columns = coordinates.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
for row in table.find_all('tr'):
    elements = row.find_all('td')
    if not elements:
        continue
    postalCode, borough, neighborhood = [element.text.strip() for element in elements]
    if borough != 'Not assigned':
        latitude = coordinates[coordinates.postal_code == postalCode].latitude.iloc[0]
        longitude = coordinates[coordinates.postal_code == postalCode].longitude.iloc[0]
        if neighborhood != 'Not assigned':
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':neighborhood, 'Latitude': latitude, 'Longitude': longitude})
        else:
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':borough, 'Latitude': latitude, 'Longitude': longitude})

## 4. Create pandas dataframe from the array

In [3]:
# instantiate the dataframe
df = pd.DataFrame(listdict)

In [4]:
group_boroughs = df.groupby('PostalCode')['Borough'].apply(lambda x: set(x).pop())
group_neighborhoods = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
group_df = pd.DataFrame(list(zip(group_boroughs.index, group_boroughs, group_neighborhoods)))
group_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
group_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data', delimiter = ',')
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
toronto_df = pd.merge(group_df, coordinates, on='PostalCode')
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [7]:
toronto_df.shape

(103, 5)

In [8]:
toronto_df.to_csv('Toronto_Neighborhood.csv', index = False)